In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.webfilters import Pisces
from mmtfPyspark.filters import ContainsLProteinChain
from mmtfPyspark.mappers import StructureToPolymerChains
from mmtfPyspark.ml import ProteinSequenceEncoder
import numpy as np
import pandas as pd
import math
import os

In [204]:
test = pd.read_csv("feature/feature-3.1.0/feature_files/4nzu.ff", sep="\t", skiprows=range(10))
test = test.reset_index()

In [205]:
cnames=[
"RESIDUE_CLASS1_IS_HYDROPHOBIC",
"RESIDUE_CLASS1_IS_CHARGED",
"RESIDUE_CLASS1_IS_POLAR",
"RESIDUE_CLASS1_IS_UNKNOWN",
"RESIDUE_CLASS2_IS_NONPOLAR",
"RESIDUE_CLASS2_IS_POLAR",
"RESIDUE_CLASS2_IS_BASIC",
"RESIDUE_CLASS2_IS_ACIDIC",
"RESIDUE_CLASS2_IS_UNKNOWN",
]
l = [["{}_SHELL{}".format(c, i) for c in cnames] for i in range(6)]
cnames = [item for sublist in l for item in sublist]
cnames.insert(0, "env")
cnames.extend(["hash", "x", "y", "z", "VERBOSITY", "location"])
test.columns = cnames
test = test.drop(["hash", "x", "y", "z", "VERBOSITY"], axis=1)

In [206]:
# filter out non-AA structures

aminoAcids = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS',
'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 
'TYR', 'VAL']

test['residue'] = test.location.str[:3]
test = test[test.residue.isin(aminoAcids)]

In [207]:
# Split identifiers

# split residue number
df_residue = test.location.str[3:].str.split(":", 1, expand=True)
df_residue.columns = ["ordernum", "chainatom"]
#df_residue.ordernum = df_residue.ordernum.str.zfill(3)
test = pd.merge(test, df_residue, left_index=True, right_index=True)
#print(df_residue.to_string())
# split chain
df_chain = test.chainatom.str.split("@", 1, expand=True)
df_chain.columns = ["chain", "atom"]
test = pd.merge(test, df_chain, left_index=True, right_index=True)

test['pdbId'] = test.env.str[4:8]

test = test.drop(["env", "location", "chainatom"], axis=1)
test
#test.ordernum = test.ordernum.astype(int)


,RESIDUE_CLASS1_IS_HYDROPHOBIC_SHELL0,RESIDUE_CLASS1_IS_CHARGED_SHELL0,RESIDUE_CLASS1_IS_POLAR_SHELL0,RESIDUE_CLASS1_IS_UNKNOWN_SHELL0,RESIDUE_CLASS2_IS_NONPOLAR_SHELL0,RESIDUE_CLASS2_IS_POLAR_SHELL0,RESIDUE_CLASS2_IS_BASIC_SHELL0,RESIDUE_CLASS2_IS_ACIDIC_SHELL0,RESIDUE_CLASS2_IS_UNKNOWN_SHELL0,RESIDUE_CLASS1_IS_HYDROPHOBIC_SHELL1,...,RESIDUE_CLASS2_IS_NONPOLAR_SHELL5,RESIDUE_CLASS2_IS_POLAR_SHELL5,RESIDUE_CLASS2_IS_BASIC_SHELL5,RESIDUE_CLASS2_IS_ACIDIC_SHELL5,RESIDUE_CLASS2_IS_UNKNOWN_SHELL5,residue,ordernum,chain,atom,pdbId
0,0,1,0,0,0,0,0,1,0,0,...,7,3,0,6,0,ASP,1,L,N,4nzu
1,0,1,0,0,0,0,0,1,0,1,...,5,5,2,5,0,ASP,1,L,CA,4nzu
2,0,2,0,0,0,0,0,2,0,2,...,7,8,2,3,0,ASP,1,L,C,4nzu
3,0,2,0,0,0,0,0,2,0,1,...,5,8,2,2,0,ASP,1,L,O,4nzu
4,0,1,0,0,0,0,0,1,0,0,...,8,5,2,3,0,ASP,1,L,CB,4nzu
5,0,2,0,0,0,0,0,2,0,0,...,9,4,0,5,0,ASP,1,L,CG,4nzu
6,0,2,0,0,0,0,0,2,0,0,...,9,1,0,3,0,ASP,1,L,OD1,4nzu
7,0,1,0,0,0,0,0,1,0,0,...,13,4,0,2,0,ASP,1,L,OD2,4nzu
8,1,0,0,0,1,0,0,0,0,3,...,11,10,2,2,0,ILE,2,L,N,4nzu
9,1,0,0,0,1,0,0,0,0,4,...,10,9,1,3,0,ILE,2,L,CA,4nzu


In [198]:
test

,env,RESIDUE_CLASS1_IS_HYDROPHOBIC_SHELL0,RESIDUE_CLASS1_IS_CHARGED_SHELL0,RESIDUE_CLASS1_IS_POLAR_SHELL0,RESIDUE_CLASS1_IS_UNKNOWN_SHELL0,RESIDUE_CLASS2_IS_NONPOLAR_SHELL0,RESIDUE_CLASS2_IS_POLAR_SHELL0,RESIDUE_CLASS2_IS_BASIC_SHELL0,RESIDUE_CLASS2_IS_ACIDIC_SHELL0,RESIDUE_CLASS2_IS_UNKNOWN_SHELL0,...,RESIDUE_CLASS2_IS_POLAR_SHELL5,RESIDUE_CLASS2_IS_BASIC_SHELL5,RESIDUE_CLASS2_IS_ACIDIC_SHELL5,RESIDUE_CLASS2_IS_UNKNOWN_SHELL5,location,residue,ordernum,chainatom,chain,atom
0,Env_4nzu_0,0,1,0,0,0,0,0,1,0,...,3,0,6,0,ASP1:L@N,ASP,1,L@N,L,N
1,Env_4nzu_1,0,1,0,0,0,0,0,1,0,...,5,2,5,0,ASP1:L@CA,ASP,1,L@CA,L,CA
2,Env_4nzu_2,0,2,0,0,0,0,0,2,0,...,8,2,3,0,ASP1:L@C,ASP,1,L@C,L,C
3,Env_4nzu_3,0,2,0,0,0,0,0,2,0,...,8,2,2,0,ASP1:L@O,ASP,1,L@O,L,O
4,Env_4nzu_4,0,1,0,0,0,0,0,1,0,...,5,2,3,0,ASP1:L@CB,ASP,1,L@CB,L,CB
5,Env_4nzu_5,0,2,0,0,0,0,0,2,0,...,4,0,5,0,ASP1:L@CG,ASP,1,L@CG,L,CG
6,Env_4nzu_6,0,2,0,0,0,0,0,2,0,...,1,0,3,0,ASP1:L@OD1,ASP,1,L@OD1,L,OD1
7,Env_4nzu_7,0,1,0,0,0,0,0,1,0,...,4,0,2,0,ASP1:L@OD2,ASP,1,L@OD2,L,OD2
8,Env_4nzu_8,1,0,0,0,1,0,0,0,0,...,10,2,2,0,ILE2:L@N,ILE,2,L@N,L,N
9,Env_4nzu_9,1,0,0,0,1,0,0,0,0,...,9,1,3,0,ILE2:L@CA,ILE,2,L@CA,L,CA


In [208]:
groups = test.groupby(['chain', 'ordernum', 'residue'], sort=False)
df_agg_max = groups.max().reset_index()
df_agg_max = df_agg_max.drop(['atom', 'ordernum'], axis=1)
df_agg_max['resi'] = df_agg_max.index+1
df_agg_max

,chain,residue,RESIDUE_CLASS1_IS_HYDROPHOBIC_SHELL0,RESIDUE_CLASS1_IS_CHARGED_SHELL0,RESIDUE_CLASS1_IS_POLAR_SHELL0,RESIDUE_CLASS1_IS_UNKNOWN_SHELL0,RESIDUE_CLASS2_IS_NONPOLAR_SHELL0,RESIDUE_CLASS2_IS_POLAR_SHELL0,RESIDUE_CLASS2_IS_BASIC_SHELL0,RESIDUE_CLASS2_IS_ACIDIC_SHELL0,...,RESIDUE_CLASS1_IS_CHARGED_SHELL5,RESIDUE_CLASS1_IS_POLAR_SHELL5,RESIDUE_CLASS1_IS_UNKNOWN_SHELL5,RESIDUE_CLASS2_IS_NONPOLAR_SHELL5,RESIDUE_CLASS2_IS_POLAR_SHELL5,RESIDUE_CLASS2_IS_BASIC_SHELL5,RESIDUE_CLASS2_IS_ACIDIC_SHELL5,RESIDUE_CLASS2_IS_UNKNOWN_SHELL5,pdbId,resi
0,L,ASP,0,2,0,0,0,0,0,2,...,6,10,0,13,8,2,6,0,4nzu,1
1,L,ILE,2,0,0,0,2,0,0,0,...,19,16,0,18,14,4,19,0,4nzu,2
2,L,GLU,0,2,0,0,0,0,0,2,...,4,16,2,8,13,4,4,0,4nzu,3
3,L,MET,2,0,0,0,2,0,0,0,...,4,27,2,17,27,5,4,0,4nzu,4
4,L,THR,0,0,2,0,0,2,0,0,...,4,16,5,11,17,0,4,0,4nzu,5
5,L,GLN,0,0,2,0,0,2,0,0,...,1,24,4,15,23,1,1,0,4nzu,6
6,L,SER,0,0,2,0,0,2,0,0,...,0,16,3,8,17,0,0,0,4nzu,7
7,L,PRO,2,0,0,0,2,0,0,0,...,3,14,2,9,15,3,0,0,4nzu,8
8,L,SER,0,0,2,0,0,2,0,0,...,4,11,2,12,12,4,0,0,4nzu,9
9,L,SER,0,0,2,0,0,2,0,0,...,9,11,2,8,11,4,6,0,4nzu,10


In [221]:
#df_agg_max.residue
print(df_agg_max[['chain','residue', 'resi']].to_string())

    chain residue  resi
0       L     ASP     1
1       L     ILE     2
2       L     GLU     3
3       L     MET     4
4       L     THR     5
5       L     GLN     6
6       L     SER     7
7       L     PRO     8
8       L     SER     9
9       L     SER    10
10      L     LEU    11
11      L     SER    12
12      L     ALA    13
13      L     SER    14
14      L     THR    15
15      L     GLY    16
16      L     ASP    17
17      L     LYS    18
18      L     VAL    19
19      L     THR    20
20      L     ILE    21
21      L     THR    22
22      L     CYS    23
23      L     GLN    24
24      L     ALA    25
25      L     SER    26
26      L     GLN    27
27      L     ASP    28
28      L     ILE    29
29      L     ALA    30
30      L     LYS    31
31      L     PHE    32
32      L     LEU    33
33      L     ASP    34
34      L     TRP    35
35      L     TYR    36
36      L     GLN    37
37      L     GLN    38
38      L     ARG    39
39      L     PRO    40
40      L     GL

In [224]:
def parse_feature_file(filename):
    # Read file
    df = pd.read_csv(filename, sep="\t", skiprows=range(10))
    df = df.reset_index()
    # Set column names
    cnames=[
    "RESIDUE_CLASS1_IS_HYDROPHOBIC",
    "RESIDUE_CLASS1_IS_CHARGED",
    "RESIDUE_CLASS1_IS_POLAR",
    "RESIDUE_CLASS1_IS_UNKNOWN",
    "RESIDUE_CLASS2_IS_NONPOLAR",
    "RESIDUE_CLASS2_IS_POLAR",
    "RESIDUE_CLASS2_IS_BASIC",
    "RESIDUE_CLASS2_IS_ACIDIC",
    "RESIDUE_CLASS2_IS_UNKNOWN",
    ]
    l = [["{}_SHELL{}".format(c, i) for c in cnames] for i in range(6)]
    cnames = [item for sublist in l for item in sublist]
    cnames.insert(0, "env")
    cnames.extend(["hash", "x", "y", "z", "VERBOSITY", "location"])
    df.columns = cnames
    
    df = df.drop(["hash", "x", "y", "z", "VERBOSITY"], axis=1)
    
    # filter out non-AA structures
    aminoAcids = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS',
    'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 
    'TYR', 'VAL']
    df['residue'] = df.location.str[:3]
    df = df[df.residue.isin(aminoAcids)]
    
    # Split identifiers

    # split residue number
    df_residue = df.location.str[3:].str.split(":", 1, expand=True)
    df_residue.columns = ["ordernum", "chainatom"]
    df = pd.merge(df, df_residue, left_index=True, right_index=True)

    # split chain
    df_chain = df.chainatom.str.split("@", 1, expand=True)
    df_chain.columns = ["chain", "atom"]
    df = pd.merge(df, df_chain, left_index=True, right_index=True)
    
    df['pdbId'] = df.env.str[4:8]

    df = df.drop(["env", "location", "chainatom"], axis=1)
    
#    df.ordernum = df.ordernum.astype(int)

    # Aggregate to residue level
    groups = df.groupby(['chain', 'ordernum', 'residue'], sort=False)
    groupnums = groups.ngroup(ascending=True)
    df_agg_max = groups.max().reset_index()
    df_agg_max = df_agg_max.drop(['atom', 'ordernum'], axis=1)
    df_agg_max['resi'] = df_agg_max.index+1
    return(df_agg_max)

In [226]:
parse_feature_file("feature/feature-3.1.0/feature_files/4nzu.ff")

,chain,residue,RESIDUE_CLASS1_IS_HYDROPHOBIC_SHELL0,RESIDUE_CLASS1_IS_CHARGED_SHELL0,RESIDUE_CLASS1_IS_POLAR_SHELL0,RESIDUE_CLASS1_IS_UNKNOWN_SHELL0,RESIDUE_CLASS2_IS_NONPOLAR_SHELL0,RESIDUE_CLASS2_IS_POLAR_SHELL0,RESIDUE_CLASS2_IS_BASIC_SHELL0,RESIDUE_CLASS2_IS_ACIDIC_SHELL0,...,RESIDUE_CLASS1_IS_CHARGED_SHELL5,RESIDUE_CLASS1_IS_POLAR_SHELL5,RESIDUE_CLASS1_IS_UNKNOWN_SHELL5,RESIDUE_CLASS2_IS_NONPOLAR_SHELL5,RESIDUE_CLASS2_IS_POLAR_SHELL5,RESIDUE_CLASS2_IS_BASIC_SHELL5,RESIDUE_CLASS2_IS_ACIDIC_SHELL5,RESIDUE_CLASS2_IS_UNKNOWN_SHELL5,pdbId,resi
0,L,ASP,0,2,0,0,0,0,0,2,...,6,10,0,13,8,2,6,0,4nzu,1
1,L,ILE,2,0,0,0,2,0,0,0,...,19,16,0,18,14,4,19,0,4nzu,2
2,L,GLU,0,2,0,0,0,0,0,2,...,4,16,2,8,13,4,4,0,4nzu,3
3,L,MET,2,0,0,0,2,0,0,0,...,4,27,2,17,27,5,4,0,4nzu,4
4,L,THR,0,0,2,0,0,2,0,0,...,4,16,5,11,17,0,4,0,4nzu,5
5,L,GLN,0,0,2,0,0,2,0,0,...,1,24,4,15,23,1,1,0,4nzu,6
6,L,SER,0,0,2,0,0,2,0,0,...,0,16,3,8,17,0,0,0,4nzu,7
7,L,PRO,2,0,0,0,2,0,0,0,...,3,14,2,9,15,3,0,0,4nzu,8
8,L,SER,0,0,2,0,0,2,0,0,...,4,11,2,12,12,4,0,0,4nzu,9
9,L,SER,0,0,2,0,0,2,0,0,...,9,11,2,8,11,4,6,0,4nzu,10
